# Prompt Chaining 模式演示

本notebook演示了Prompt Chaining（提示链）模式，这是一种将多个LLM调用串联起来的技术。
每个模型的输出作为下一个模型的输入，形成一个处理链条。

**工作流程：**
1. LLM-1 (gpt-5-mini) 生成初始内容
2. 过滤函数处理LLM-1的输出
3. LLM-2 (claude-sonnet-4-5) 接收过滤后的内容并进行处理
4. LLM-3 (deepseek) 接收LLM-2的输出并生成最终结果

## 1、加载密钥环境OpenRouter

In [1]:
import os
import json
import re
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

load_dotenv(override=True)
api_key = os.getenv("OPENROUTER_API_KEY")
if api_key:
    print("API Key loaded successfully.")
else:
    print("Failed to load API Key.")

API Key loaded successfully.


## 2、初始化OpenRouter客户端

In [ ]:
openrouter = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)
print("OpenRouter客户端初始化成功")

## 3、定义过滤函数

这个函数用于处理第一个LLM的输出，提取关键信息并格式化，为第二个LLM做准备。
在实际应用中，这个函数可以执行各种处理：
- 提取关键信息
- 格式化输出
- 数据清洗
- 结构化转换

In [ ]:
def filter_and_process(text):
    """
    过滤和处理LLM-1的输出
    
    参数:
        text: 第一个LLM的原始输出
    
    返回:
        处理后的文本，用于传递给下一个LLM
    """
    # 移除多余的空行
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # 提取关键句子（这里简单地按句号分割并过滤短句）
    sentences = [s.strip() for s in text.split('。') if len(s.strip()) > 10]
    
    # 统计信息
    word_count = len(text)
    sentence_count = len(sentences)
    
    # 构建过滤后的输出
    filtered_output = {
        "original_length": word_count,
        "sentence_count": sentence_count,
        "key_sentences": sentences[:5],  # 只保留前5个关键句子
        "summary": text[:200] + "..." if len(text) > 200 else text  # 摘要
    }
    
    print("\n=== 过滤函数处理结果 ===")
    print(f"原始文本长度: {word_count} 字符")
    print(f"提取句子数量: {sentence_count} 句")
    print(f"保留关键句子: {len(filtered_output['key_sentences'])} 句")
    
    return filtered_output

## 4、第一步：LLM-1 生成初始内容

使用 **gpt-5-mini** 模型生成关于某个主题的初始内容

In [ ]:
# 定义初始提示
initial_prompt = "请写一篇关于人工智能在医疗领域应用的简短文章，包括当前的应用场景、面临的挑战和未来的发展方向。"

print("=== 第一步：LLM-1 (gpt-5-mini) 生成初始内容 ===")
print(f"输入提示: {initial_prompt}\n")

# 调用第一个模型
model_1 = "gpt-5-mini"
messages_1 = [{"role": "user", "content": initial_prompt}]

response_1 = openrouter.chat.completions.create(
    model=model_1,
    messages=messages_1
)

output_1 = response_1.choices[0].message.content
print(f"\n模型: {model_1}")
print("="*50)
display(Markdown(output_1))

## 5、第二步：过滤和处理LLM-1的输出

使用自定义过滤函数处理第一个模型的输出，提取关键信息

In [ ]:
print("\n=== 第二步：过滤处理 ===")
filtered_data = filter_and_process(output_1)

print("\n过滤后的数据结构:")
print(json.dumps(filtered_data, ensure_ascii=False, indent=2))

## 6、第三步：LLM-2 处理过滤后的内容

使用 **anthropic/claude-sonnet-4-5** 模型对过滤后的内容进行深度分析

In [ ]:
print("\n=== 第三步：LLM-2 (claude-sonnet-4-5) 深度分析 ===")

# 构建第二个模型的提示
prompt_2 = f"""基于以下关于AI医疗应用的内容摘要和关键句子，请进行深度分析：

内容摘要：
{filtered_data['summary']}

关键句子：
{chr(10).join([f"{i+1}. {s}" for i, s in enumerate(filtered_data['key_sentences'])])}

请从以下角度进行分析：
1. 技术可行性评估
2. 潜在的伦理问题
3. 实施建议

请提供结构化的分析报告。"""

print(f"输入提示长度: {len(prompt_2)} 字符\n")

model_2 = "anthropic/claude-sonnet-4-5"
messages_2 = [{"role": "user", "content": prompt_2}]

response_2 = openrouter.chat.completions.create(
    model=model_2,
    messages=messages_2
)

output_2 = response_2.choices[0].message.content
print(f"\n模型: {model_2}")
print("="*50)
display(Markdown(output_2))

## 7、第四步：LLM-3 生成最终输出

使用 **deepseek/deepseek-chat** 模型基于LLM-2的分析生成最终的行动计划

In [ ]:
print("\n=== 第四步：LLM-3 (deepseek) 生成最终行动计划 ===")

# 构建第三个模型的提示
prompt_3 = f"""基于以下关于AI医疗应用的深度分析报告：

{output_2}

请制定一份具体的、可执行的行动计划，包括：
1. 短期目标（6个月内）
2. 中期目标（1-2年）
3. 长期愿景（3-5年）
4. 关键里程碑
5. 资源需求
6. 风险缓解措施

请以清晰、专业的格式呈现这份行动计划。"""

print(f"输入提示长度: {len(prompt_3)} 字符\n")

model_3 = "deepseek/deepseek-chat"
messages_3 = [{"role": "user", "content": prompt_3}]

response_3 = openrouter.chat.completions.create(
    model=model_3,
    messages=messages_3
)

output_3 = response_3.choices[0].message.content
print(f"\n模型: {model_3}")
print("="*50)
display(Markdown(output_3))

## 8、总结：Prompt Chaining 流程回顾

展示整个链式调用的流程和每个阶段的输出摘要

In [ ]:
print("\n" + "="*70)
print("Prompt Chaining 流程总结")
print("="*70)

summary = f"""
### 链式调用流程：

**阶段1 - 内容生成 (gpt-5-mini)**
- 输入: 原始提示词
- 输出长度: {len(output_1)} 字符
- 任务: 生成AI医疗应用的初始文章

**阶段2 - 数据过滤**
- 输入: LLM-1的完整输出
- 处理: 提取关键句子、生成摘要
- 输出: 结构化的过滤数据

**阶段3 - 深度分析 (claude-sonnet-4-5)**
- 输入: 过滤后的关键信息
- 输出长度: {len(output_2)} 字符
- 任务: 从技术、伦理、实施角度进行分析

**阶段4 - 行动计划 (deepseek)**
- 输入: LLM-2的分析报告
- 输出长度: {len(output_3)} 字符
- 任务: 制定具体可执行的行动计划

### Prompt Chaining 的优势：

1. **任务分解**: 将复杂任务分解为多个简单步骤
2. **专业化**: 每个模型专注于特定任务
3. **质量控制**: 中间过滤步骤确保数据质量
4. **灵活性**: 可以根据需要调整链条中的任何环节
5. **可追溯**: 每个阶段的输出都可以被检查和验证
"""

display(Markdown(summary))

## 9、可视化链式调用流程

In [ ]:
flow_diagram = """
```
┌─────────────────────────────────────────────────────────────┐
│                    Prompt Chaining 流程图                    │
└─────────────────────────────────────────────────────────────┘

    ┌──────────────────┐
    │   用户输入提示    │
    └────────┬─────────┘
             │
             ▼
    ┌──────────────────┐
    │   LLM-1 (GPT)    │  ◄── 生成初始内容
    │   gpt-5-mini     │
    └────────┬─────────┘
             │
             ▼
    ┌──────────────────┐
    │   过滤函数        │  ◄── 提取关键信息
    │  filter_process  │      格式化数据
    └────────┬─────────┘
             │
             ▼
    ┌──────────────────┐
    │  LLM-2 (Claude)  │  ◄── 深度分析
    │ claude-sonnet-4-5│
    └────────┬─────────┘
             │
             ▼
    ┌──────────────────┐
    │ LLM-3 (DeepSeek) │  ◄── 生成行动计划
    │  deepseek-chat   │
    └────────┬─────────┘
             │
             ▼
    ┌──────────────────┐
    │   最终输出结果    │
    └──────────────────┘
```
"""

display(Markdown(flow_diagram))

## 10、保存完整的链式调用结果

In [ ]:
# 将所有结果保存到一个字典中
chain_results = {
    "stage_1": {
        "model": model_1,
        "input": initial_prompt,
        "output": output_1,
        "output_length": len(output_1)
    },
    "stage_2": {
        "process": "filter_and_process",
        "filtered_data": filtered_data
    },
    "stage_3": {
        "model": model_2,
        "output": output_2,
        "output_length": len(output_2)
    },
    "stage_4": {
        "model": model_3,
        "output": output_3,
        "output_length": len(output_3)
    }
}

# 保存到JSON文件
with open('prompt_chaining_results.json', 'w', encoding='utf-8') as f:
    json.dump(chain_results, f, ensure_ascii=False, indent=2)

print("✓ 链式调用结果已保存到 prompt_chaining_results.json")
print(f"\n总共执行了 3 个LLM调用 + 1 个过滤处理")
print(f"总输出字符数: {len(output_1) + len(output_2) + len(output_3)} 字符")